# Mapping Yelp Search Results - Part 1

- 04/26/22


## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [38]:
# !pip install yelpapi
!pip install tqdm

In [39]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [40]:
# Load API Credentials
with open('/Users/codingdojo/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [41]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s=5)
yelp

In [42]:
# help(yelp)

### Define Search Terms and File Paths

In [43]:
# set our API call parameters and filename before the first call
LOCATION = "Baltimore, MD, 21202"
TERM = "Burgers"

In [44]:
## Specify folder for saving data
FOLDER = "Data/"
os.makedirs(FOLDER , exist_ok=True)

In [45]:
LOCATION.split(',')[0]

'Baltimore'

In [46]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = f"{FOLDER}{TERM}_{LOCATION.split(',')[0]}.json"
JSON_FILE

'Data/Burgers_Baltimore.json'

### Check if Json File exists and Create it if it doesn't

In [47]:
os.path.dirname(JSON_FILE)

'Data'

In [48]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    ## saving empty file
    with open(JSON_FILE,'w') as f:
        json.dump([], f)
    

        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Burgers_Baltimore.json already exists.


### Load JSON FIle and account for previous results

In [49]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)

## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [50]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term=TERM, location=LOCATION)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [51]:
## How many results total?
results['total']

530

- Where is the actual data we want to save?

In [52]:
type(results['businesses'])

list

In [53]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [54]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(results['total'] / results_per_page)
n_pages

27

In [55]:
for i in tqdm_notebook( range(0,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
        ## Read in results in progress file and check the length
        ## Load previous results and use len of results for offset
        with open(JSON_FILE,'r') as f:
            previous_results = json.load(f)

        ## set offset based on previous results
        n_results = len(previous_results)
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(term=TERM, 
                                    location=LOCATION,
                                   offset=n_results+1)
        
        
        

        

        ## append new results and save to file
        previous_results.extend( results['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results, f)
    ## What to do if we get an error/exception.
    except Exception as e:
        print(e)
        break

  0%|          | 0/28 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [56]:
df = pd.read_json(JSON_FILE)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,914,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"{'latitude': 39.2771835822848, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1041 Marshall St', 'address2': '...",+14434539698,(443) 453-9698,2161.212353
1,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media1.fl.yelpcdn.com/bphoto/mmtgGp...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,32,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 39.32696, 'longitude': -76.63753}","[delivery, pickup]",$$,"{'address1': '3300 Clipper Mill Rd', 'address2...",,,4242.702639
2,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,135,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 39.27174, 'longitude': -76.60223}","[delivery, pickup]",$$,"{'address1': '823 E Fort Ave', 'address2': '',...",+14106373699,(410) 637-3699,2754.995467
3,jvKhto6__tCGfs2N27oWdg,the-outpost-american-tavern-baltimore-2,The Outpost American Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/4bN4vA...,False,https://www.yelp.com/biz/the-outpost-american-...,183,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 39.2774761241692, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1032 Riverside Ave', 'address2':...",+14433889113,(443) 388-9113,2083.657647
4,5CM8nuqd0od68GqQ-UGicQ,cookhouse-baltimore-2,CookHouse,https://s3-media2.fl.yelpcdn.com/bphoto/N-2JWj...,False,https://www.yelp.com/biz/cookhouse-baltimore-2...,20,"[{'alias': 'newamerican', 'title': 'American (...",5.0,"{'latitude': 39.30715667, 'longitude': -76.625...","[delivery, pickup]",NaN,"{'address1': '1501 Bolton St', 'address2': Non...",+14102259964,(410) 225-9964,1990.078956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 39.33086, 'longitude': -76.63161}","[delivery, pickup]",$$,"{'address1': '907 W 36th St', 'address2': None...",+14108893663,(410) 889-3663,4382.210159
525,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,231,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,"{'latitude': 39.2821578979492, 'longitude': -7...",[delivery],$,"{'address1': '1720 Thames St', 'address2': '',...",+14105580399,(410) 558-0399,2079.005086
526,vnOzM6USxuhmuXV_uqg0JA,filippos-baltimore,Filippo's,https://s3-media2.fl.yelpcdn.com/bphoto/NiWpYq...,False,https://www.yelp.com/biz/filippos-baltimore?ad...,51,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 39.28699, 'longitude': -76.56748}","[delivery, pickup]",$,"{'address1': '418 S Conkling St', 'address2': ...",+14103274550,(410) 327-4550,3607.859236
527,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,"{'latitude': 39.322087392211, 'longitude': -76...",[delivery],NaN,"{'address1': '3320 Belair Rd', 'address2': '',...",+14105636222,(410) 563-6222,4134.435594


In [58]:
df.duplicated(subset='id').sum()

0

In [59]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Burgers_Baltimore.csv.gz'

In [60]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [63]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(csv_file)

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz:.2f} times smaller!')

JSON FILE: 519,369 Bytes
CSV.GZ FILE: 73,272 Bytes
the csv.gz is 7.09 times smaller!


## Next Class: Processing the Results and Mapping 